# <div align="center"> SPECIAL TOPICS III </div>
## <div align="center"> Data Science for Social Scientists  </div>
### <div align="center"> ECO 4199 </div>
#### <div align="center">Class 9A - Model Selection</div>
<div align="center"> Jonathan Holmes, (he/him)</div>

# Subset Selection
$$Y = \beta_0+\beta_1X_1+\beta_2X_2+ ... + \beta_pX_p + \epsilon$$
- The issue it that adding predictors will always weakly improve the in sample prediction
    - But at the expense of out of sample prediction
- It is therefore important to limit the number of predictors to those actually related to Y
- __Subset Selection__ _is the process of identifying the p predictors actually related to Y._

## Best Subset Selection
- Here is the algorithm to select the best subset given a dataset with p predictors
- We will apply this algorithm to the Credit dataset

Algorithm:

1. Let $\mathscr{M}_0$ denote the null model , which contains no predictors. This
model simply predicts the sample mean for each observation.
2. For k = 1, 2, . . .p:
    - a. Fit all ${P\choose k}$ containing exactly $k$ predictors
    - b. Pick the best among these ${P\choose k}$ and call it $\mathscr{M}_k$. Here best is defined as having the smallest RSS, or equivalently largest R2.
3. Select a single best model from among $\mathscr{M}_0$, . . . ,$\mathscr{M}_p$ using crossvalidated prediction error (MSE), AIC, BIC, or adjusted $R^2$

### The null model
- Let's start with the null model
- This a model using no predictors and a single parameter: the intercept

In [ ]:
# Let me set my current directory using the %cd magic
%cd "~/Dropbox/_teaching/ECO4199/2023/Data-Science-for-Social-Scientists/Class 09A – Model Selection/"

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import statsmodels.api as sm
import statsmodels.formula.api as smf

import sklearn.linear_model as skl_lm
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

from itertools import permutations

In [ ]:
df=pd.read_csv("Credit.csv")
df['Own']=df['Own'].replace({"No":0,  "Yes":1})
df['Student']=df['Student'].replace({"No":0,  "Yes":1})
df['Married']=df['Married'].replace({"No":0,  "Yes":1})
df=df.join(pd.get_dummies(df['Region'], drop_first=True))
df.pop("Region")
display(df.shape)
df.head()

$\mathscr{M}_0$

In [ ]:
# M0
results = smf.ols('Balance ~ 1 ', data=df).fit()
# Inspect the results
print(results.summary())

In [ ]:
# get y and X
y=df['Balance'] # y
X=df.loc[:,~df.columns.str.contains('Balance')] # X
X.head()

$\mathscr{M}_1$

In [ ]:
# initiate empty list to store R2
R2= 0
best_predictor="none"
X=df.loc[:,~df.columns.str.contains('Balance')]
y=df.loc[:,df.columns.str.contains('Balance')]
X['intercept']=1


# for each predictor in X
for k in range(X.shape[1]-1): # minus 1 because last column is intercept
    results = sm.OLS(y, X.iloc[:,[k,-1]]).fit() # fit predictor in position p and intercept (in last position, -1)
    print(f"R\u00b2 for predictor {X.columns[k]} is {round(results.rsquared,6):f}")
    if results.rsquared>R2:
        R2=results.rsquared ; best_predictor=X.columns[k]
print("\nBest predictor for M1 is: {} with {}.".format(best_predictor,round(R2,3)))

# In-Class Exercise: 

Q1: Which single variable does the best job at predicting "Balance"? 

Q2: How many possible combinations are there if you have: 
- 1 variable: 
- 2 variables: 
- 3 variables: 
- 4 variables: 
- 5 variables: 


In [ ]:
DFs=[]
#for p in range(X.shape[1]): # uncomment here if you want to do it for all RUN AT YOUR OWN RISK
for p in range(8):
    R2=0
    best_predictor="none"
    R2_list=[]; adj_R2_list=[] ; aic=[] ; bic=[] ; models=[]
    bestmodel=[-1]
    for k in permutations(list(range(X.shape[1]-1)), p):
        model=list(k)
        model.append(-1)
        results = sm.OLS(y, X.iloc[:,model]).fit() # fit predictors from model
        # append statistics to list
        R2_list.append(results.rsquared) ; adj_R2_list.append(results.rsquared_adj) 
        aic.append(results.aic); bic.append(results.bic) ; models.append(','.join(X.columns[model]))

        if results.rsquared_adj>R2:
            bestmodel=model
            #model.pop(-1)
            R2=results.rsquared_adj 
    DFs.append(pd.DataFrame(data={'R2':R2_list,'Adjusted R2':adj_R2_list, "AIC":aic, "BIC":bic,'Predictors':p,'model':models}))
    
    print("Best predictor for M{} is: {} with {}.".format(p,', '.join(X.columns[bestmodel]),round(R2,3)))
    
dd=pd.concat(DFs, ignore_index=True)
dd.head()

## Computational challenge
- Best subset selection is a simple and conceptually appealing 
- But the number of possible models that must be considered grows rapidly as p increases. 
- In general, there are 2p models that involve subsets of p predictors. 
    - So if p = 10, then there are approximately 1,000 possible models to be considered!
    - So if p = 20, then there are more than 1,000,000 possible models to be considered!!!

![title](credit_10predictors.png)

In [ ]:
dd_best=dd.groupby(['Predictors']).agg({"R2":np.max, "Adjusted R2":np.max, "AIC":np.min, "BIC":np.min}).reset_index()
dd_best

In [ ]:
fig, axes = plt.subplots(2,2, figsize=(10,10),sharex=True)
axes = axes.ravel() # access axes with a single position instead of 2
for i, statistics in enumerate(['R2',"Adjusted R2","AIC","BIC"]):
    sns.scatterplot(x='Predictors',y=statistics,data=dd,ax=axes[i], color='gray',marker='.',alpha=.3)
    sns.lineplot(x='Predictors',y=statistics,data=dd_best,ax=axes[i], color='darkorange')
    sns.scatterplot(x='Predictors',y=statistics,data=dd_best,ax=axes[i], color='darkgreen')
    axes[i].set_ylabel(statistics)
    axes[i].set_xticks(np.arange(p+1))
    
fig.suptitle("In-Sample Statistics")
fig.tight_layout()
plt.show()

### Step 3 of Best Subset Selection
- You can think of this stage in terms of last lecture:
    - You have many models (last time many polynomial models), one for each p
    - But they all maximize the in sample fit
- In step 3, you can do this based on the adjusted-$R^2$, AIC, BIC or you can use CV techniques to find the model that gives the __best out of sample prediction__

In [ ]:
dd['best']=dd.groupby(['Predictors'])['Adjusted R2'].transform(np.max)
dd_best2=dd.loc[dd['Adjusted R2']==dd['best']].groupby('Predictors').first().reset_index()
dd_best2.head(10)

In [ ]:
models=["intercept",["Rating","intercept"], 
        ["Income","Rating","intercept"],
        ["Income","Rating","Student","intercept"],
        ["Income","Limit","Cards","Student","intercept"],
        ["Income","Limit","Rating","Cards","Student","intercept"],
        ["Income","Limit","Rating","Cards","Age","Student","intercept"],
        ["Income","Limit","Rating","Cards","Age","Own","Student","intercept"]]

In [ ]:
df['intercept']=1

In [ ]:
# Best number of predictors using Cross Validation
# use best model from precedent exercise to speed up the code
kfold=5
DFs=[]
kf = KFold(n_splits=kfold, random_state=1706, shuffle=True)

for i,m in enumerate(models):
    MSEs=[] # empty list of MSE scores
    X=df[m].values
    y=df['Balance'].values
    for train_index, test_index in kf.split(X):
        if i==0:
            X_train, X_test = X[train_index].reshape(-1, 1), X[test_index].reshape(-1, 1)
        else:
            X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        # regression
        reg = LinearRegression() # initiate the regression class
        reg.fit(X_train,y_train) # fit the data
        # Out of Sample MSE:
        mse=mean_squared_error(y_test, reg.predict(X_test))
        MSEs.append(mse)
    DFs.append(pd.DataFrame({'Predictors':i,'MSE':MSEs}))
    
MSE_scores=pd.concat(DFs)
mse=MSE_scores.groupby('Predictors').mean().reset_index()    
mse

In [ ]:
dd_best=dd_best.merge(mse, on='Predictors')
dd_best.head()

In [ ]:
# Best number of predictors for other statistics
adj_R2_best=int(dd_best2.loc[dd_best2['Adjusted R2']==dd_best2['Adjusted R2'].max(),'Predictors'])
aic_best=int(dd_best2.loc[dd_best2['AIC']==dd_best2['AIC'].min(),'Predictors'])
bic_best=int(dd_best2.loc[dd_best2['BIC']==dd_best2['BIC'].min(),'Predictors'])
mse_best=int(mse.loc[mse.MSE==mse.MSE.min(),'Predictors'])
best_preds=[adj_R2_best,aic_best,bic_best,mse_best]
print(f"Best number of parameters for\nAdjusted R-square: {adj_R2_best}\nAIC: {aic_best}\nBIC: {bic_best}\n10-fold CV:{mse_best} ")

fig, axes = plt.subplots(2,2, figsize=(12,12),sharex=True)
axes = axes.ravel() # access axes with a single position instead of 2
for i, statistics in enumerate(["Adjusted R2","AIC","BIC", "MSE"]):
    sns.lineplot(x='Predictors',y=statistics,data=dd_best,ax=axes[i], color='darkorange')
    sns.scatterplot(x='Predictors',y=statistics,data=dd_best,ax=axes[i], color='darkgreen')
    # axes[i].axvline(best_preds[i], color='k')
    axes[i].scatter(x=best_preds[i],y=float(dd_best.loc[best_preds[i],statistics]),marker='X',color='red',s=100)
    axes[i].set_ylabel(statistics)
    axes[i].set_xticks(np.arange(p+1))
fig.suptitle("Best number of parameters by technique")
plt.show()

## Solutions to computational challenge: Forward Stepwise Selection

1. Let $\mathscr{M}_0$ denote the null model, which contains no predictors.
2. For $k = 0, \dots , p − 1$:
    - (a) Consider all p − k models that augment the predictors in $\mathscr{M}_k$ with one additional predictor.
    - (b) Choose the best among these p − k models, and call it $\mathscr{M}_{k+1}$.
        - Here best is defined as having smallest RSS or highest R2.
3. Select a single best model from among $\mathscr{M}_0$, $\dots$ ,$\mathscr{M}_p$ using crossvalidated prediction error, AIC, BIC, or adjusted R2.

## Solutions to computational challenge: Backward Stepwise Selection

1. Let $\mathscr{M}_p$ denote the full model, which contains all p predictors.
2. For $k = p, p − 1, \dots , 1$:
    - (a) Consider all k models that contain all but one of the predictors in $\mathscr{M}_k$, for a total of k − 1 predictors.
    - (b) Choose the best among these k models, and call it $\mathscr{M}_{k-1}$. 
        - Here best is defined as having smallest RSS or highest $R^2$.
3. Select a single best model from among $\mathscr{M}_0, \dots ,\mathscr{M}_p$ using crossvalidated prediction error, AIC, BIC, or adjusted R2.

# When to use forward VS backward stepwise selection? 

Advantage of forward stepwise selection: 
- If your dataset has many, many variables forward stepwise selection is easier/feasible

Advantage of backward stepwise selection: 
- It works better with collinear variables







## Taking Stock
- Last week we saw the risks associated with overfitting
- This risk increases with the number of parameters
- Different techniques yield different types of subsets but all penalize, one way or another, having too many parameters
- The trade-off therefore is to find the best out of sample prediction using the least number of predictors possible

# Shrinkage Methods
- Think back on the credit dataset
- The data set has a number of predictors, which all seem reasonable
    - All seem to be legitimate predictors of Balance
    - There no variable irrelevant variable
- Instead of our iterative, and long, process it would be nicer to fit all p predictors using a technique that __constrains__ or __regularizes the coefficient estimates__, or equivalently, that shrinks the coefficient estimates towards zero. 
    - Instead of cherry picking parameters we, instead, force the parameters of redundant predictors to be small or zero

# Ridge Regression
- OLS regression for a model with p parameters finds that $\beta_0, \beta_1, ... \beta_p$ that minimize (as you know):
$$\Large \text{RSS} = \sum_{i=1}^n \Big(y_i - \beta_0 - \sum_{j=1}^p \beta_jx_{ij}\Big)^2$$


## Ridge Regression, continued
- Ridge regression is very similar to least squares, except that the coefficients ridge are by minimizing a slightly different quantity. 
- In particular, the ridge regression coefficient estimates $\hat{\beta^R}$ are the values that minimize:
\begin{gather}
\Large \sum_{i=1}^n \Big(y_i - \beta_0 - \sum_{j=1}^p \beta_jx_{ij}\Big)^2 + \lambda \sum_{j=1}^p\beta_j^2 \\
= \Large\text{RSS} + \lambda \sum_{j=1}^p\beta_j^2
\end{gather}


## Ridge Regression, shrinkage
- $\lambda$ is a __tuning parameter__ that is determined outside of the minimization problem
- As with OLS, Ridge regression seeks coefficient estimates that fit the data well (small RSS)
- Unlike OLS, the second term $\lambda \sum_{j=1}^p\beta_j^2$ is small if $\beta_1, ..., \beta_j$ are small ($\beta_0$ not included!)
    - This second term is known as a __shrinkage penalty__
- The tuning parameter $\lambda$ serves to control the relative impact of these two terms on the regression coefficient estimates.
    - When $\lambda$ = 0, the penalty term has no effect, and ridge regression = OLS
    - $\lambda \to \infty$, the impact of the shrinkage penalty grows, and the ridge regression coefficient estimates will approach zero.
- As you will see, the optimal $\lambda$ is given using cross validation

## Standardization of your dataset
- Standardization of datasets is a common requirement for many machine learning estimators implemented see this tutorial in [scikit-learn](https://scikit-learn.org/stable/modules/preprocessing.html)

- You can transform the data to center it by removing the mean value of each feature, then scale it by dividing non-constant features by their standard deviation.

- Models such as the Ridge regression assume that all features are centered around zero and have variance in the same order. 
- If a feature has a variance that is orders of magnitude larger than others, it might dominate the objective function and make the estimator unable to learn from other features correctly as expected.
- In other words, in the OLS regression, multiplying X by a constant c will change $\beta$ to $\frac{\beta}{c}$
- In the Ridge regression, the $\beta^R$ will depend not only on the value of λ, but also on the scaling of the $j^{th}$ predictor

# In-Class Exercise

Q3: Why do we call "Ridge" regression a "Shrinkage" estimator? 

Q4: Suppose that we are predicting a farm harvest using the following regression

$$\text{Total_Harvest} = \beta_0 + \beta_1\text{Average_Temperature} + \beta_2\text{Average_Rainfall} + u $$
    
a) If I run OLS two times with temperature measured in degrees celcius and degrees kelvin, what will change? 

b) If I ran Ridge regression two times measured in degrees celcius and degrees kelvin, what will change? 

    



In [ ]:
df.head()

In [ ]:
# import the preprocessing module from sklearn
from sklearn import preprocessing
X_train =df[['Income','Limit','Rating','Student','Cards','Age','Education','Married','Own', 'West', 'South']]
scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_train.mean(axis=0) , X_train.std(axis=0)

Let's now see what happens to our estimates as the value for $\lambda$ changes

In [ ]:
from sklearn.linear_model import Ridge

lambdas = 10**np.linspace(10,-2,100)*0.5
ridge = Ridge()
coefs = []

for 𝜆 in lambdas:
    ridge.set_params(alpha=𝜆)
    ridge.fit(X_train, y)
    coefs.append(ridge.coef_)
ridge_results=pd.DataFrame(coefs,columns=['Income','Limit','Rating','Student','Cards','Age','Education','Married','Own', 'West', 'South'])
ridge_results['Lambda']=lambdas  
ridge_results=pd.melt(ridge_results,id_vars=['Lambda'], var_name='Beta', value_name='Estimate')
ridge_results.head()                        


In [ ]:
fig, ax =plt.subplots(1,1, figsize=(7,7))
sns.lineplot(x='Lambda', y='Estimate', hue='Beta',data=ridge_results)
ax.set_xscale('log')
ax.axhline(0,color='k',linestyle=":")
plt.axis('tight')
plt.xlabel(r'$\lambda$', fontsize=20)
plt.ylabel('Standardized Coefficients',fontsize=20)
plt.title('Ridge coefficients as a function of the regularization');

plt.title('Plot B', fontsize=20)

## Ridge and Model Selection
- The ridge regression is clearly faster than our best subset methodology
- Note though that we never truly select a model in the sense that we never use a subset of predictors
- Instead, we are shrinking how much they matter in our prediction but use all p predictors (unless λ = ∞).
- This may not be a problem for prediction accuracy, but it can create a challenge in model interpretation in settings in which the number of variables p is quite large. 
- For example, in the Credit data set, it appears that the most important variables are income, limit, rating, and student. 
- So we might wish to build a model including just these predictors. 

## Lasso Regression
- The Lasso regression is an alternative to the Ridge regression as it allows to shrink parmeters to zero
- The lasso regression coefficient estimates $\hat{\beta^L_\lambda}$ are the values that minimize:
\begin{gather}
\Large\text{RSS} + \lambda \sum_{j=1}^p\left|\beta_j\right|
\end{gather}

## Lasso and Model Selection
- As with ridge regression, the lasso shrinks the coefficient estimates towards zero. 
- However, in the case of the lasso, the $\ell_1$ penalty has the effect of forcing some of the coefficient estimates to be exactly equal to zero when the tuning parameter λ is sufficiently large. 
- Hence, much like best subset selection, the lasso performs variable selection.
- We say that the lasso yields __sparse models__ 

In [ ]:
from sklearn.linear_model import Lasso

lambdas = 10**np.linspace(10,-1.5,100)*0.5
lasso = Lasso()
coefs = []


for 𝜆 in lambdas:
    lasso.set_params(alpha=𝜆)
    lasso.fit(X_train, y)
    coefs.append(lasso.coef_)
lasso_results=pd.DataFrame(coefs,columns=['Income','Limit','Rating','Student','Cards','Age','Education','Gender','Married','Asian','Caucasian'])
lasso_results['Lambda']=lambdas  
lasso_results=pd.melt(lasso_results,id_vars=['Lambda'], var_name='Beta', value_name='Estimate')
lasso_results.head()                        


In [ ]:
fig, ax =plt.subplots(1,1, figsize=(7,7))
ax = plt.gca()
sns.lineplot(x='Lambda', y='Estimate', hue='Beta',data=lasso_results)
ax.set_xscale('log')
ax.axhline(0,color='k',linestyle=":")
plt.axis('tight')
plt.xlabel(r'$\lambda$', fontsize=20)
plt.ylabel('Standardized Coefficients',fontsize=20)
#plt.title('Lasso coefficients as a function of the regularization');

plt.title('Plot A', fontsize=20)

## Another Formulation for Ridge Regression and the Lasso
- You may have recognized something you are already familiar with as economists
- The Lasso and Ridge regressions can be written in terms of objective function (to minimize) and a constraint
 
 __Ridge__:
 \begin{gather}
 \large \min_{\mathbf{\beta}} \left\{ \sum_{i=1}^n \Big(y_i - \beta_0 - \sum_{j=1}^p \beta_jx_{ij}\Big)^2 \right\} \\ \large \text{subject to } \ \sum_{j=1}^p\beta_j^2 \leq s
 \end{gather}
 
 __Lasso__:
  \begin{gather}
 \large \min_{\mathbf{\beta}} \left\{ \sum_{i=1}^n \Big(y_i - \beta_0 - \sum_{j=1}^p \beta_jx_{ij}\Big)^2 \right\} \\ \large \text{subject to } \ \sum_{j=1}^p\left|\beta_j\right| \leq s
 \end{gather}


## Another Formulation for Ridge Regression and the Lasso, continued

- The formulas in the previous slide mean that for every value of λ, there is some s such that the constraint minization yields the same result as our first definition of Lasso and Ridge
- When p=2 (2 predictors): 
    - the lasso coefficient estimates have the smallest RSS such that $\left|\beta_1\right|+ \left|\beta_2\right| \leq s $
    - the ridge coefficient estimates have the smallest RSS such that $\beta_1^2 + \beta_2^2 \leq s$

- We can think of it as follows.
    - When we perform the Lasso or Ridge we are trying to find the set of coefficient estimates that lead to the smallest RSS, subject to the constraint that there is a budget s for how large $\sum_{j=1}^p \left|\beta_j\right|$ or 
$\sum_{j=1}^p \beta_j^2$ can be.

- If s is large the restriction is not binding (not restrictive)
- For s large enough you get the OLS estimates (which are unconstrained)

![title](lasso_ridge.png)

## OLS vs Ridge and Lasso
- In the previous slide, the OLS solution is marked as $\hat{\beta}$ and lies outside the constraint
- If s was sufficiently large, Ridge and Lasso estimates would be the same as OLS (case where $\lambda=0$)
- The ellipses that are centered around $\hat{\beta}$ represent regions of constant RSS. 
- As the ellipses expand away from the least squares coefficient estimates, the RSS increases. 
- The lasso and ridge regression coefficient estimates are given by the first point at which an ellipse contacts the constraint region. 
- Since ridge regression has a circular constraint with no sharp points, this intersection will not generally occur on an axis, and so the ridge regression coefficient estimates will be exclusively non-zero. 
- However, the lasso constraint has corners at each of the axes, and so the ellipse will often intersect the constraint region at an axis. When this occurs, one of the coefficients will equal
     - Here, the intersection occurs at β1 = 0, and so theresulting model will only include β2.

## Selecting the tuning parameter
- Since a lot seems to depend on the value of $\lambda$ which value should you choose?
- As usual, we need to remember that our end goal is to maximize out of sample prediction
- As such the right model and/or the right tuning parameter will be given by cross validation

![image.png](lasso_lambda.png)

# In-Class Exercise

Q5: What is the difference between Lasso and Ridge regression? 

Q6: How could we decide which model to use (Lasso, Ridge, or OLS)? 


In [ ]:
#Here is an example of using built-in Python commands to do cross-validated Lasso and Ridge

from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV

Xdata = X_train
#Xdata = X


results_lasso = LassoCV(cv=10, random_state=31415).fit(Xdata, y)

print("Lasso CV selected lambda = {} with an $R^2$ of {}".format(
    round(results_lasso.alpha_,2), 
    round(results_lasso.score(Xdata,y), 2)))

#print(results_lasso.coef_)
                  

results_ridge = RidgeCV(cv=10).fit(Xdata, y)


print("Ridge CV selected lambda = {} with an $R^2$ of {}".format(
    round(results_ridge.alpha_,2), 
    round(results_ridge.score(Xdata,y), 2)))

#print(results_ridge.coef_)
    
    
    
    